### TEXT PREPROCESSING

In [1]:
import nltk
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from ast import literal_eval
import re, string
import pandas as pd


df =  pd.read_csv("dataset.txt")
tweets = df['tweet'].tolist()

def preprocess_word(w):
    # Removes punctuation
    
    translator = str.maketrans('', '', string.punctuation)
    punctuation = w.translate(translator)

    return punctuation


def preprocessing(x):
    # Returns a nested list of the processed sentences
    
    mentions = [re.sub(r'@\w+',"", sent) for sent in tweets] #removes mentions
    numbers = [re.sub('[0-9]+', "", sent) for sent in mentions] #removes numbers
    links = [re.sub(r'http\S+', "", sent) for sent in numbers] #removes links
    
    lower = [[sent.lower()] for sent in links] #lower text
    in_list = [word for sent in lower for word in sent]
    word_tokenized = [word_tokenize(sent) for sent in in_list]
    word_tokenized = [sent for sent in word_tokenized if sent] #word tokenization
    
    for _id, sent in enumerate(word_tokenized):
        word_tokenized[_id] =  [preprocess_word(w) for w in sent]

    words = [[word for word in sent if word != '' and word != 'rt' and len(word)>1] for sent in word_tokenized] #removes useless words
    sentences = [sent for sent in words if sent] #removes empty sentences

    return sentences

text = preprocessing(tweets)
print(len(text))

286


### BAG OF WORDS 

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

try: 
    assert(literal_eval(str(text)) == text.copy())
except AssertionError:
    print('failed to convert')
    
final_str = ([" ".join(x) for x in text])

count_vect = CountVectorizer()
bow = count_vect.fit_transform(final_str).toarray()
print(len(bow))

286


In [3]:
vocab = count_vect.get_feature_names()
print(len(vocab))

1371


In [4]:
#Returns the frequency of every word in total
sumindex = [sum(x) for x in zip(*bow)]
print(len(sumindex))

1371


### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = count_vect.fit_transform(final_str).toarray()
print(tfidf)
tfidf.shape

### EMBEDDINGS 

In [5]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np

model = Word2Vec(sentences=text, window=5, min_count=1, workers=4)
model.save("word2vec.model")

vector = model.wv['no'] #returns numpy vector of a word
sims = model.wv.most_similar('no', topn=10) #returns similar words

embeddings = [model.wv[word] for word in text]

C:\Users\Angelina\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Calculate the word vector average for every sentence:

In [6]:
v_average = []
for i in text:
    av = np.mean(model.wv[i], axis=0)
    v_average.append(av)

In [ ]:
from sklearn.decomposition import IncrementalPCA  
from sklearn.manifold import TSNE                 
import numpy as np                                  


def reduce_dimensions(model):
    num_dimensions = 2 


    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key) 

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(autosize=True)
    plt.scatter(x_vals, y_vals)

    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

### SYNTAX

In [ ]:
def flatten_list(x):
#Takes a nested list and converts it into a list of elements
#where every sublist is a new element

    new_list = [] 
    
    for sent in x:
        sentences = " ".join(sent)
        new_list.append(sentences)
    
    return new_list

In [ ]:
import spacy

def dependency_parsing(x):
# Returns a nested list of syntactic labels

    nlp = spacy.load("en_core_web_sm")
    dependencies = []
    for sent in x:
        doc = nlp(sent)
        new_list = [token.dep_ for token in doc]
        dependencies.append(new_list)
        
    return dependencies

dep = dependency_parsing(new)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import string
import pprint
%matplotlib inline

wordset = set()
for sentence in text:
    for word in sentence:
        wordset.add(word)

nlp = spacy.load("en_core_web_sm")

#Add every word as a node
base_graph = nx.Graph()
base_graph.add_nodes_from(wordset)
nx.draw(base_graph, with_labels=True)

graphs = {}
for sentence_id, sentence_contents in enumerate(new):
    sentence_graph = base_graph.copy()
    processed_sentence =  nlp(' '.join(new))
print('\n',processed_sentence)

In [ ]:
#Add edges between the nodes according to syntactic relations
for token in processed_sentence:
    nodeA = token.text
    nodeB = token.head.text
    #print('\tadding edge between', nodeA, 'and', nodeB)
    sentence_graph.add_edge(nodeA, nodeB)
    sentence_representation =  nx.adjacency_matrix(sentence_graph) #sparse matrix
    #print('\t sparse matrix has ',sentence_representation.count_nonzero(),'nonzero elements')
    graphs[sentence_id] = sentence_representation

In [ ]:
options = {
    "font_size": 20,
    "node_size": 30,
    "node_color": "white",
    "edgecolors": 'blue',
    "linewidths": 2,
    "width": 2,
}
plt.figure(3,figsize=(33,33)) 
nx.draw(sentence_graph, with_labels=True, **options)

In [ ]:
#print(sentence_representation)

In [ ]:
print('\n\nsaving representations...')
import pickle
with open('sentence_representations_adjmat.p','wb') as fw:
    pickle.dump(graphs, fw)

#### DataFrame

In [ ]:
A = pd.DataFrame(columns=['Tweets','Class','BoW','Embeddings'])
A['Tweets'] = [sent for sent in text]

In [ ]:
A['Class'] = df['class']
print(A)

#### Frequency & Embeddings lists

In [ ]:
wfrequency_list = []
for sent in text:
    for word in sent:
        ind = vocab.index(word)
        #print('\n',word, '\nIndex:', ind)
        freq = sumindex[ind]
        #print("BOW frequency:", freq)
        wfrequency_list.append(freq)

In [ ]:
wembeddings_list = []
for l in embeddings:
    for subl in l:
        wembeddings_list.append(subl)

### CLASSIFICATION

#### Classification using Bag-of-Words:

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [19]:
x = bow
y = df['class'].astype(int)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
logr = LogisticRegression()
logr.fit(x_train, y_train)
bow_predictions = logr.predict(x_test)
print(bow_predictions)

[1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]


In [23]:
bow_report = classification_report(y_test, bow_predictions)
print(bow_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.82      0.98      0.89        55
           2       0.83      0.33      0.48        15

    accuracy                           0.82        72
   macro avg       0.55      0.44      0.46        72
weighted avg       0.80      0.82      0.78        72



#### Classification using embeddings:

In [24]:
v_train, v_test, y_train, y_test = train_test_split(v_average, y, test_size=0.25, random_state=0)
logr.fit(v_train, y_train)
emb_predictions = logr.predict(v_test)
print(emb_predictions)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [26]:
emb_report = classification_report(y_test, emb_predictions)
print(emb_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.76      1.00      0.87        55
           2       0.00      0.00      0.00        15

    accuracy                           0.76        72
   macro avg       0.25      0.33      0.29        72
weighted avg       0.58      0.76      0.66        72



#### Classification using both Bag-of-Words and embeddings: 

In [31]:
conc = np.concatenate([bow, v_average], axis=1)

In [32]:
c_train, c_test, y_train, y_test = train_test_split(conc, y, test_size=0.25, random_state=0)
logr.fit(c_train, y_train)
bow_emb_predictions = logr.predict(c_test)
print(bow_emb_predictions)

[1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]


In [33]:
bow_emb_report = classification_report(y_test, bow_emb_predictions)
print(bow_emb_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.82      0.98      0.89        55
           2       0.83      0.33      0.48        15

    accuracy                           0.82        72
   macro avg       0.55      0.44      0.46        72
weighted avg       0.80      0.82      0.78        72



C:\Users\Angelina\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
